In [35]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [36]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
import requests

In [37]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()
results = search_tool.invoke("What is today's top news")
results


'Read full articles, watch videos, browse thousands of titles and more on the "Headlines" topic with Google News . 11 hours ago · Maharashtra Breaking News Today : LIVE Updates on OCTOBER 16 महत्त्वाच्या ब्रेकिंग न्यूज मराठीमध्ये, महाराष्ट्रातील सर्व घडामोडी पाहा ( Maharashtra LIVE News ) एका क्लिकवर, मुंबई ... 20 hours ago · ABC News Live VideoNY Gov. Hochul holds news conference on transit security grants Read the latest headlines , breaking news , and videos at APNews.com, the definitive source for independent journalism from every corner of the globe. Lokmat Times Offers Latest News and Breaking News Updates & Information from India & World related to Politics, Sports, Entertainment, business, technology, Lifestyle, and more'

In [38]:
llm=ChatOpenAI(
    model = 'gpt-4o-mini',
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url = os.getenv("OPENAI_API_BASE")
)

llm.invoke('HI').content

'Hello! How can I assist you today?'

In [39]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

In [40]:
# pull react prompt from the langchain hub

prompt = hub.pull('hwchase17/react') # pulls the standard react agent
#ReAct means reasoning and action which is a design pattern


In [41]:
agent = create_react_agent(
    llm = llm,
    tools = [search_tool],
    prompt = prompt
)

In [42]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool],
    verbose=True
)

In [43]:
response = agent_executor.invoke({"input":"3 ways to reach mumbai from Goa"})



> Entering new AgentExecutor chain...
I need to find information about transportation options from Goa to Mumbai. This might include flights, trains, or bus services.  
Action: duckduckgo_search  
Action Input: "ways to reach Mumbai from Goa"  September 4, 2025 - NH48 (Mumbai–Pune–Kolhapur–Goa) – Faster, with better highways. ... Distance: 590 km. Time: 10–12 hours. Fuel cost: ₹3,500–₹4,000 one way (petrol car). May 6, 2025 - It takes an hour to reach Goa by flight. The Goa International Airport is located at Dabolim. Whilst this is probably the quickest mode of transport, flights are not the most sustainable way to travel. ... Getting from Mumbai to Goa by train is relatively cheaper. December 16, 2024 - Mumbai to Goa are well connected via flights . The nearest major airport to Mumbai is Chhatrapati Shivaji International Airport Mumbai (BOM / VABB). This airport is 16 km from the centre of Mumbai. One can book an airport taxi in Mumbai to reach there on time. December 10, 2024 - We

In [44]:
response["output"]

'The three ways to reach Mumbai from Goa are by air (flight), by train, and by road (car or bus).'

In [45]:
# @tool

# def get_weather(city:str) -> str:
#     """
#     this function gives the weather report of the city which is asked about.
#     """

#     url = f"http://api.weatherstack.com/current?access_key=428f04ae72fa809f015fd14efdbf9a2e&query={city}"

#     response = requests.get(url)
#     return response.json()

from langchain.tools import tool

@tool("Get the current weather of a city using a weather API")
def get_weather(city: str) -> str:
    """
    Returns the current temperature, humidity, and general weather conditions of the given city.
    """
    import requests
    url = f"http://api.weatherstack.com/current?access_key=428f04ae72fa809f015fd14efdbf9a2e&query={city}"
    response = requests.get(url)
    data = response.json()
    temp = data['current']['temperature']
    humidity = data['current']['humidity']
    condition = data['current']['weather_descriptions'][0]
    return f"The temperature is {temp}°C, humidity is {humidity}%, and it is {condition.lower()}."


In [46]:
new_prompt = hub.pull('hwchase17/react')

In [47]:
new_agent = create_react_agent(
    llm = llm,
    tools=[get_weather],
    prompt = new_prompt
)

In [ ]:
new_agent_executor = AgentExecutor(
    agent = new_agent,
    tools = [get_weather],
    verbose = True
)

In [49]:
response = agent_executor.invoke({"input":"Tell me about the weather of the Mumbai"})
print(response)



> Entering new AgentExecutor chain...
I should check the current weather conditions in Mumbai for the most accurate information. 
Action: duckduckgo_search
Action Input: Mumbai current weatherHourly Local Weather Forecast, weather conditions, precipitation, dew point, humidity, wind from Weather .com and The Weather Channel Mumbai expects a sunny day with a gentle breeze, though humidity will make it feel warm. Yesterday's 'poor' air quality (AQI 156) might persist, advis Mumbai Weather Update: Rains, Temperature, and Air Quality Outlook Stay updated with today's Mumbai weather : overcast skies, light rain showers, temperatures between 26°C and 30°C, 88% humidity ... Get the latest hourly weather updates for Mumbai today. Detailed forecast including temperature, wind, rain, snow, and UV index. Stay informed about today's weather conditions in Mumbai . 12 day Mumbai Weather Forecast. Live Weather Warnings, hourly weather updates. Accurate Mumbai weather today, forecast for sun, rain, 

In [50]:
# its not using the get_weather but it can, this is due to the prebuilt prompt pulled from hub, it has not set the search_tool as a general purpose tool
# and so it uses it for mostly everything